In [3]:
from rouge import Rouge, FilesRouge
import pandas as pd
import numpy as np

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)

In [4]:
print(scores)

[{'rouge-1': {'f': 0.49411764217577864, 'p': 0.5833333333333334, 'r': 0.42857142857142855}, 'rouge-2': {'f': 0.23423422957552154, 'p': 0.3170731707317073, 'r': 0.18571428571428572}, 'rouge-l': {'f': 0.42751590030718895, 'p': 0.5277777777777778, 'r': 0.3877551020408163}}]


In [5]:
d = {'pred': [hypothesis], 'actuals': [reference]}

In [24]:
# read in actual notes
notes = pd.read_csv("Z:/final_data/scored_data/notes_scored.csv",index_col=0)
notes.columns = ['scored_summaries']

In [25]:
notes.head()

,scored_summaries
18,Increase in diffuse pulmonary opacities which ...
19,Increase in diffuse pulmonary opacities which ...
20,Diffuse nonspecific interstitial disease of un...
21,Diffuse nonspecific interstitial disease of un...
22,Right chest tube in place with no significant ...


In [13]:
# join notes to the test set
test_dta = pd.read_csv("Z:/final_data/cohorts_merged_test.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (11,18,21,35,54,63,66,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185,187,188,190,191,193,194,196,197,199,200,202,203,205,206,208,209,211,212,214,215,217,218,220,221,223,224,226,227,229,230,232,233,235,236,238,239,241,242,244,245,247,248,250,251,253,254,256,257,259,260,262,263,265,266,268,269,271,272,274,275,277,278,280,281,283,284,328,329,330,331,332,333,334,335,336,337) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
test_dta.head(n=20)

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death,encounter_id,enc_eio,visit_status_1,visit_status_2,...,technique,comparison,findings,impressions,elecsig,patient_raw,rawtext,encounter_id_diagnosed,date_diagnosed,ild_status
0,49,79.0,M,White,Not Hispanic or Latino,NaN,10440586,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,49,79.0,M,White,Not Hispanic or Latino,NaN,10440586,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,49,79.0,M,White,Not Hispanic or Latino,NaN,10440586,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,49,79.0,M,White,Not Hispanic or Latino,NaN,10440586,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,49,79.0,M,White,Not Hispanic or Latino,NaN,11420608,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,49,79.0,M,White,Not Hispanic or Latino,NaN,11420608,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,49,79.0,M,White,Not Hispanic or Latino,NaN,11420608,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,49,79.0,M,White,Not Hispanic or Latino,NaN,11420608,O,No Show,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,49,79.0,M,White,Not Hispanic or Latino,NaN,11455337,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,104,84.0,M,White,Not Hispanic or Latino,NaN,9339110,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [27]:
# join in the columns 
# test_dta.join(notes)
scored_df = pd.merge(test_dta, notes, left_index=True, right_index=True,how="left")

In [31]:
# test df
rouge.get_scores(scored_df.scored_summaries,scored_df.findings)

AttributeError: 'float' object has no attribute 'split'

In [75]:
type(notes.pred)

pandas.core.series.Series

In [94]:
def test_rouge(df):
    for index,row in df.iterrows():
        print(row)
        print(row['pred'],row['actuals'])
        rougescore = rouge.get_scores(row['pred'],row['actuals'])
        print(rougescore)

In [95]:
test_rouge(notes)

pred       the #### transcript is a written version of ea...
actuals    this page includes the show transcript use the...
Name: 0, dtype: object
the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teac    her or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests     students ' knowledge of even ts in the news
[{'rouge-1': {'f': 0.49411764217577864, 'p': 0.5833333333333334, 'r': 0.42857142857142855}, 'rouge-2': {'f': 0.23423422957552154, 'p': 0.3170731707317073, 'r': 0.18571428571428572}, 'rouge-l': {'f': 0.4275

In [84]:
# now more rows

In [89]:
d2 = {'pred': [hypothesis,hypothesis], 'actuals': [hypothesis,reference]}
notes2 = pd.DataFrame(data=d2,columns=["pred","actuals"])

In [91]:
notes2

,pred,actuals
0,the #### transcript is a written version of ea...,the #### transcript is a written version of ea...
1,the #### transcript is a written version of ea...,this page includes the show transcript use the...


In [96]:
test_rouge(notes2)

pred       the #### transcript is a written version of ea...
actuals    the #### transcript is a written version of ea...
Name: 0, dtype: object
the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news
[{'rouge-1': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}, 'rouge-2': {'f': 0.999999995, 'p': 1.0, 'r': 1.0}, 'rouge-l': {'f': 0.9999999999995, 'p': 1.0, 'r': 1.0}}]
pred       the #### transcript is a written version of ea...
actuals    this page includes the show transcript use the...
Name: 1, dtype: object
the #### transcript is a written version

In [308]:
notes2

,pred,actuals
0,the #### transcript is a written version of ea...,the #### transcript is a written version of ea...
1,the #### transcript is a written version of ea...,this page includes the show transcript use the...


# now let's get a full function to take any given df, define rows of pred and actuals and score them, and append to a df

In [32]:
def score_data(df, predicted, actuals):
    for index,row in df.iterrows():
#         print(row)
        print("[info] Predicted")
        print(row[predicted])
        print("[info] Predicted")
        print(row[actuals])
        rougescore = rouge.get_scores(row[predicted],row[actuals])
        tmpdf = pd.DataFrame(rougescore)
        print("[info] now appending to list with reshape")
        for row in tmpdf:
            scorelist.append(tmpdf[row].apply(pd.Series).values[0])

In [360]:
# get np array now of rouge scores
scorelist = []
score_data(notes2,'pred','actuals')

[info] Predicted
the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news
[info] Predicted
the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news
[info] now appending to list with reshape
[info] Predicted
the #### transcript is a written version of each day 's cnn student news program use this transcript to he    lp students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you     saw on cnn student news
[info] Predicted
this page includes the show transcript use the transcript to help students with reading comprehension and     vocabulary at t

In [363]:
# assign
score_df = pd.DataFrame(np.array(scorelist).reshape(int(len(scorelist)/3),9),columns=['rouge-1-f','rouge-1-p','rouge-1-r','rouge-2-f','rouge-2-p','rouge-2-r','rouge-l-f','rouge-2-p','rouge-2-r'])
score_df.head()

,rouge-1-f,rouge-1-p,rouge-1-r,rouge-2-f,rouge-2-p,rouge-2-r,rouge-l-f,rouge-2-p,rouge-2-r
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,0.494118,0.583333,0.428571,0.234234,0.317073,0.185714,0.427516,0.527778,0.387755
